In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/.MAIN

Mounted at /content/drive
/content/drive/My Drive/MSc_project/.MAIN


## Breakdown of the number of MS and Single-Source cases

In [ ]:
import json 
import os
from types import new_class
import pandas as pd
from copy import deepcopy
import tweepy
import time
from datetime import datetime
import os.path

def load_new_informer_df():
    path = f'{hashtag}{os.path.sep}updated_informers_data.json'
    with open(path) as jf:
        data = json.load(jf)
    return data

In [ ]:
hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']
# hashtags = ['blm']

SSR = {}
SSF = {}

MSR = {}
MSF = {}

all_breakdown = {}

inf_friends = {}

for hashtag in hashtags:
    # print(f'\n{hashtag}')

    data = load_new_informer_df()

    df = pd.DataFrame.from_dict(data,orient='index')

    # # SINGLE SOURCE CASES
    # ssr = df [ (df['num-informers'] == 1) & (df['infector-is-friend'] == 1)] 
    # ssf = df [ (df['num-informers']) == 1 & (df['infector-is-friend'] == 0)] 
    # # MULTI SOURCE CASES
    # msr = df [ (df['num-informers'] > 1) & (df['infector-is-friend'] == 1)] 
    # msf = df [ (df['num-informers'] > 1) & (df['infector-is-friend'] == 0)] 
    # num_inf = df['infector-is-friend'].sum()

    # # update info
    # SSR.update( {hashtag:len(ssr)})
    # SSF.update( {hashtag:len(ssf)})
    # MSR.update( {hashtag:len(msr)})
    # MSF.update( {hashtag:len(msf)})
    # inf_friends.update( {hashtag:num_inf} )

    all_ms = df [ df['infector-is-friend'] == 1 ]

    # BREAKDOWN OF THE NUMBER OF INFORMERS A TWEET HAS!!!
    # breakdown = pd.DataFrame(all_ms['num-informers']).sort_values(by=['num-informers'])
    breakdown = pd.DataFrame(all_ms['num-informers'].value_counts() ).sort_index()

    info = pd.DataFrame(breakdown.iloc[0:10])

    info.loc[len(info)+1] = {'num-informers':breakdown.iloc[10:20].sum().values[0]}
    info.loc[len(info)+1] = {'num-informers':breakdown.iloc[20:40].sum().values[0]}
    info.loc[len(info)+1] = {'num-informers':breakdown.iloc[40:80].sum().values[0]}
    info.loc[len(info)+1] = {'num-informers':breakdown.iloc[80:].sum().values[0]}

    # path = f'informer_results{os.path.sep}number_of_informers_breakdown{os.path.sep}{hashtag}_breakdown.csv'
    # info.to_csv(path)

    all_breakdown.update( {hashtag:info.to_dict()['num-informers']})


In [ ]:
# all_breakdown
df = pd.DataFrame.from_dict(all_breakdown, orient='columns')
print(df)

    avengers   blm  borisjohnson  brexit  climatechange  covid  gaza  \
1        572  1738           604     154            636    950   587   
2        253   746           351     147            354    352   327   
3        167   414           230     115            231    272   213   
4        147   303           146      82            190    157   131   
5        112   229           116      90            141    142   119   
6         76   178            87      90            103    101    83   
7         70   123            69      62             80     71    75   
8         57   128            75      64             65     60    44   
9         38    99            56      52             63     64    45   
10        32   120            57      58             46     59    38   
11       145   502           359     366            308    300   248   
12       130   462           472     441            184    166   116   
13        85   356           425     503            113     76  

In [ ]:
df.to_csv(f'informer_results{os.path.sep}number_of_informers_breakdown{os.path.sep}all_breakdown_numbers.xls')

In [ ]:
df = pd.DataFrame.from_dict(MSR, orient='index',columns = ['Multi-Source w Infct-Friend'])
df.rename_axis('hashtag',axis=1)
df['Multi-Source no Infct-Friend'] = df.index.map(MSF)
df['Single-Source w Infct-Friend'] = df.index.map(SSR)
df['Single-Source no Infct-Friend'] = df.index.map(SSF)
df['Infector-Friends'] = df.index.map(inf_friends)

In [ ]:
df
df.to_csv(f'informer_results{os.path.sep}poo.xls')

## Multi-Purpose Class for extracting Info

In [ ]:
# Load in the friends_db
import json
import os
import os.path

def get_friends_db():
    jsons = [pos_json for pos_json in os.listdir(f'user-friends{os.path.sep}') if pos_json.endswith('.json')]
    all_js = {}
    for file in jsons:
        with open(os.path.join(f'user-friends{os.path.sep}' + file)) as jf:
            all_js = { **all_js, **json.load(jf) }
    print(f'pulled data on {len(all_js)} users')
    return all_js

friends_db = get_friends_db()

In [ ]:
## Part of stage 2 to grab the necessary info of the multi-source case.
# grab the base information of the infectors of the tweet.

import json 
import os
from types import new_class
import pandas as pd
from copy import deepcopy
import tweepy
import time
from datetime import datetime
import os.path

class crawler(object):
    def __init__(self, hashtag, friends_db): 

        hashtag = hashtag
        self.sep = os.path.sep
        self.friends_db = friends_db
    ################################################################################################################
    ################################################################################################################
    ################################################################################################################


    @staticmethod
    def process_loaded_all(tweet):
        d = {'id':tweet['id_str'], 'tweet-text': tweet['full_text'],'created_at': tweet['created_at'], 'user-id':tweet['user']['id_str'],
        'location': tweet['user']['location'],
        'num-followers': tweet['user']['followers_count'], 'num-following':  tweet['user']['friends_count'], 'description': tweet['user']['description'],
        'retweet_count': tweet['retweet_count'], 'favourite_count':tweet['favorite_count']}
        return d

    @staticmethod
    def process_loaded_targets(tweet):
        d = {'id':tweet['id_str'], 'tweet-text': tweet['full_text'],'created_at': tweet['created_at'], 'user-id':tweet['user']['id_str'],
        'check-idx':None,'informers-data':None,'friend-ids':None, 'location': tweet['user']['location'],
        'num-followers': tweet['user']['followers_count'], 'num-following':  tweet['user']['friends_count'], 'description': tweet['user']['description'],
        'retweet_count': tweet['retweet_count'], 'favourite_count':tweet['favorite_count']}
        return d



####################################################################################

# FUNCTINOS INVOLVING GETTING THE TWEETS WITH HASHTAGS

####################################################################################

    def load_in_tweets(self):
        path = f'{self.hashtag}{os.path.sep}{self.hashtag}_all_tweets.json'
        with open(path) as jf:
            data = json.load(jf)
        return data

    def load_informer_df(self):
        path = f'{self.hashtag}{os.path.sep}informer_database.json'
        with open(path) as jf:
            data = json.load(jf)
        return data

    def load_new_informer_df(self):
        path = f'{self.hashtag}{os.path.sep}updated_informers_data.json'
        with open(path) as jf:
            data = json.load(jf)
        return data



    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################
        '''

        THE MAIN FUNCTION!!!!!!!!!!!!!!!!!!!!!!!!
        
        '''
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################

    def infector_extraction(self):


        # WHEN LOADING TWEETS IN
        tweet_list = self.load_in_tweets()
        ########################################################

        informer_df = self.load_informer_df()

        dic=  {}
        for tweet in tweet_list:
            dic.update({tweet['id_str']:tweet})
        
        # print('turn list to dic')

        new_dic = {}

        # get the data of the informer-db
        for key, value in informer_df.items():
            value['num-informers'] = len(value['informers-data'])
            try: 
                infector_data = dic[key]['retweeted_status']
                value['infector-info'] = {infector_data['user']['id_str']:self.process_loaded_all(infector_data)}
                if infector_data['user']['id'] in value['friend-ids']:
                    value['infector-is-friend'] = 1
                else: 
                    value['infector-is-friend'] = 0
                new_dic.update( {key:value})
            except:
                4+4
                # print(f'source was not a retwet. delete from multi-source df')
        # print('added infector data of all')

        path = f'{self.hashtag}{os.path.sep}updated_informers_data.json'
        with open(path, 'w') as fp:
            json.dump(informer_df,fp)

    def get_num_of_cases_info(self):
        print(f'\n{self.hashtag}')

        data = self.load_new_informer_df()

        df = pd.DataFrame.from_dict(data,orient='index')

        # SINGLE SOURCE CASES
        single_source = df [ df['num-informers'] == 1 ] 
        print(f'SINGLE SOURCE CASES: {len(single_source)}')

        # infector is friend
        ms_cases = df['infector-is-friend'].sum()
        print(f'INFECTOR IS FRIEND CASES: {ms_cases}. Out of {len(df)}')


        # BREAKDOWN OF THE NUMBER OF INFORMERS A TWEET HAS!!!
        breakdown = pd.DataFrame(df['num-informers'].value_counts(normalize=True) * 100).sort_index()

        info = pd.DataFrame(breakdown.iloc[0:10])

        info.loc[len(info)+1] = {'num-informers':breakdown.iloc[10:20].sum().values[0]}
        info.loc[len(info)+1] = {'num-informers':breakdown.iloc[20:40].sum().values[0]}
        info.loc[len(info)+1] = {'num-informers':breakdown.iloc[40:80].sum().values[0]}
        info.loc[len(info)+1] = {'num-informers':breakdown.iloc[80:].sum().values[0]}

        path = f'{self.hashtag}{os.path.sep}{self.hashtag}_breakdown.csv'
        info.to_csv(path)

    def info_of_all(self):

        # WHEN LOADING TWEETS IN
        tweet_list = self.load_in_tweets()

        dic=  {}
        for tweet in tweet_list:
            dic.update({tweet['id_str']:tweet})

        # collect all retweets
        self.all_retweets = {tweet['id']:self.process_loaded_targets(tweet) for tweet in tweet_list if "RT @" in tweet['full_text'] }

        # Find the number of retweets that were retweeted from a friend

        print('num of retweets in data')
        print(len(self.all_retweets))
        users_done_data = self.friends_db

        users_done_ids = list( users_done_data.keys() )

        current_ids = [ value['user-id'] for _,value in self.all_retweets.items() ]

        already_done = list(set(users_done_ids).intersection(current_ids))

        targets_with_friends = {}

        if already_done: # have users info in our database!!!!

            print(f'we already have data for {len(already_done)} users!! Removing them from API search')

            targets_without_friends = deepcopy(self.all_retweets)

            all_retweets_df = pd.DataFrame.from_dict(self.all_retweets, orient='index')

            for user_id in already_done:

                # entries with those user ids
                entries = all_retweets_df.loc[ all_retweets_df['user-id'] == user_id].to_dict(orient='index')
                for key in entries:
                    del targets_without_friends[key]

                for key in entries:
                    entries[key]['friend-ids'] = users_done_data[user_id]['friend-ids']

                targets_with_friends.update(entries)

            print(f'num of tweets w friends {len(targets_with_friends)}')
            print(f'num of tweets w no friends {len(targets_without_friends)}')

        else:
            print(f'oops we got no data on this batch')
            targets_without_friends = self.all_retweets

        
        # now have all the users with frien-ids

        for key, value in targets_with_friends.items():
            try:
                infector_data = dic[str(key)]['retweeted_status']
                value['infector-info'] = {infector_data['user']['id_str']:self.process_loaded_all(infector_data)}
                if infector_data['user']['id'] in value['friend-ids']:
                    value['infector-is-friend'] = 1
                else: 
                    value['infector-is-friend'] = 0
            except:
                3+3

        data = targets_with_friends

        df = pd.DataFrame.from_dict(data,orient='index')
        

        # infector is friend
        print(self.hashtag)
        ms_cases = df['infector-is-friend'].sum()
        print(f'INFECTOR IS FRIEND CASES: {ms_cases}. Out of {len(df)}\n')




In [ ]:
# hashtags = ['avengers']
# hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']
hashtags = ['Casemiro']

for hashtag in hashtags:

    c = crawler(hashtag, friends_db)

    c.infector_extraction()
    # c.get_num_of_cases_info()
    # c.info_of_all()